In [5]:
import os
import h5py
import numpy as np
from osgeo import gdal, osr

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire'

# Path to the HDF file
hdf_file = os.path.join(maindir,'Aim2/data/spatial/raw/test/MOD14A2.A2023033.h09v05.006.2023041233546.hdf')
print(hdf_file)

/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/test/MOD14A2.A2023033.h09v05.006.2023041233546.hdf


In [6]:
# Open the HDF file using gdal
hdf_ds = gdal.Open(hdf_file, gdal.GA_ReadOnly)

/opt/miniconda3/envs/aspen-fire/lib/python3.10/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [7]:
# List all subdatasets in the HDF file
subdatasets = hdf_ds.GetSubDatasets()
print("Subdatasets in the HDF file:")
for i, subdataset in enumerate(subdatasets):
    print(f"{i}: {subdataset[0]}")

Subdatasets in the HDF file:
0: HDF4_EOS:EOS_GRID:"/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/test/MOD14A2.A2023033.h09v05.006.2023041233546.hdf":MODIS_Grid_8Day_Fire:FireMask
1: HDF4_EOS:EOS_GRID:"/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/test/MOD14A2.A2023033.h09v05.006.2023041233546.hdf":MODIS_Grid_8Day_Fire:QA


In [8]:
# Select the subdataset you want to read (example: 'FireMask')
subdataset_name = subdatasets[0][0]
subdataset = gdal.Open(subdataset_name)

# Read the data into a numpy array
data = subdataset.ReadAsArray()

# Get geo-information from the subdataset
geo_transform = subdataset.GetGeoTransform()
projection = subdataset.GetProjection()

# Get dimensions of the dataset
rows, cols = data.shape

# Create a GeoTIFF file
output_tiff = os.path.join(maindir,'Aim2/data/spatial/raw/test/output_fire_data.tif')
driver = gdal.GetDriverByName('GTiff')
out_raster = driver.Create(output_tiff, cols, rows, 1, gdal.GDT_Int16)

# Set the geo-transform and projection
out_raster.SetGeoTransform(geo_transform)
out_raster.SetProjection(projection)

# Write the data to the GeoTIFF
out_band = out_raster.GetRasterBand(1)
out_band.WriteArray(data)

# Flush data to disk and close the file
out_band.FlushCache()
out_raster = None

print(f"GeoTIFF file saved as {output_tiff}")

GeoTIFF file saved as /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/test/output_fire_data.tif


In [11]:
import h5py
import numpy as np
from osgeo import gdal, osr

# Path to the HDF5 file
hdf5_file = os.path.join(maindir,'Aim2/data/spatial/raw/test/VNP10A1.A2024069.h16v07.002.2024070100852.h5')

# Open the HDF5 file using h5py
hdf5 = h5py.File(hdf5_file, 'r')

# List all datasets in the HDF5 file
print("Datasets in the HDF5 file:")
# Function to recursively print the structure of the HDF5 file
def print_structure(name, obj):
    print(name)
# Print the structure of the HDF5 file
hdf5.visititems(print_structure)

Datasets in the HDF5 file:
HDFEOS
HDFEOS/ADDITIONAL
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES
HDFEOS/GRIDS
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/Data Fields
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/Data Fields/Algorithm_bit_flags_QA
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/Data Fields/Basic_QA
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/Data Fields/NDSI
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/Data Fields/NDSI_Snow_Cover
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/Data Fields/Projection
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/Data Fields/granule_pnt
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/XDim
HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/YDim
HDFEOS INFORMATION
HDFEOS INFORMATION/StructMetadata.0


In [13]:
# Select the dataset you want to read
dataset_name = 'HDFEOS/GRIDS/VIIRS_Grid_IMG_2D/Data Fields/NDSI_Snow_Cover'
data = hdf5[dataset_name][()]

# Extract the StructMetadata.0 attribute
gridmeta = hdf5['HDFEOS INFORMATION/StructMetadata.0'][()].decode('utf-8')

# Improved parsing for the StructMetadata.0 attribute
def find_coords(metadata, corner):
    start = metadata.find(corner + ' = (') + len(corner) + 4
    end = metadata.find(')', start)
    return metadata[start:end]

ulc_str = find_coords(gridmeta, 'UpperLeftPointMtrs')
lrc_str = find_coords(gridmeta, 'LowerRightMtrs')
x_start, y_start = map(float, ulc_str.split(','))
x_end, y_end = map(float, lrc_str.split(','))

# Calculate pixel size
rows, cols = data.shape
pixel_size_x = (x_end - x_start) / cols
pixel_size_y = (y_start - y_end) / rows  # note y_start > y_end for North-Up images

# Define the geo-transform
geo_transform = [x_start, pixel_size_x, 0, y_start, 0, -pixel_size_y]

# Get the projection information (assuming WGS84, EPSG:4326)
projection = 'EPSG:4326'

# Create a GeoTIFF file
output_tiff = os.path.join(maindir,'Aim2/data/spatial/raw/test/output_snow_cover.tif')
driver = gdal.GetDriverByName('GTiff')
out_raster = driver.Create(output_tiff, cols, rows, 1, gdal.GDT_Int16)

# Set the geo-transform and projection
out_raster.SetGeoTransform(geo_transform)
srs = osr.SpatialReference()
srs.ImportFromEPSG(int(projection.split(':')[1]))
out_raster.SetProjection(srs.ExportToWkt())

# Write the data to the GeoTIFF
out_band = out_raster.GetRasterBand(1)
out_band.WriteArray(data)

# Flush data to disk and close the file
out_band.FlushCache()
out_raster = None

print(f"GeoTIFF file saved as {output_tiff}")

ValueError: could not convert string to float: 'END_GROUP=SwathStructure\nGROUP=GridStructure\n\tGROUP=GRID_1\n\t\tGridName="VIIRS_Grid_IMG_2D"\n\t\tXDim=3000\n\t\tYDim=3000\n\t\tUpperLeftPointMtrs=(-2223901.039333'